In [ ]:
#Yelp 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn import svm
import scipy.sparse as sp
import re


In [46]:
#load reviews
train = pd.read_csv('yelp_data/yelp_2013_train.csv')
valid = pd.read_csv('yelp_data/yelp_2013_dev.csv')
test = pd.read_csv('yelp_data/yelp_2013_test.csv')

#add review length to data frame
#yelp['text length'] = yelp['text'].str.len()
#random sample for testing purppses
#yelp = yelp.sample(frac=.1)

X_train = train['text'] #.replace('\[a-zA-Z]{2}','', regex=True)
y_train = train['rating']
X_valid = valid['text']
y_valid = valid['rating']
X_test = test['text']
y_test = test['rating']

X_full = X_train + X_valid


In [45]:
'''
X_words = []
for i in range(X_train.shape[0]) : 
    words = []
    x = str(X_train[i])
    x = x.split()
    for st in x :
        if len(st) > 2 :
        #print(st)
            X_words.append(st)
            
    #text = ' '.join(words) 
    #text = np.array(text)
    #X_words.append(words)
        

#print(text)

print(X_words[0])
'''

"\nX_words = []\nfor i in range(X_train.shape[0]) : \n    words = []\n    x = str(X_train[i])\n    x = x.split()\n    for st in x :\n        if len(st) > 2 :\n        #print(st)\n            X_words.append(st)\n            \n    #text = ' '.join(words) \n    #text = np.array(text)\n    #X_words.append(words)\n        \n\n#print(text)\n\nprint(X_words[0])\n"

In [4]:
#X_words = np.array(X_words, dtype=object)
#df = pd.DataFrame({'col':X_words})
#X_words = df['col']

In [47]:
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)
train = []
valid = []
test = []

(268013,)
(33501,)
(33504,)


In [48]:
max_vocab = 50000
bow_transformer = CountVectorizer(max_features=max_vocab, ngram_range=(1,3), 
                                  binary=True ).fit(X_full.values.astype('U'))


In [5]:
print(bow_transformer.vocabulary_)

{'place': 31796, 'good': 16579, 'years': 49708, 'ago': 2056, 'worst': 49461, 'valley': 47876, 'chips': 7747, 'bad': 3993, 'came': 6626, 'table': 44068, 'looked': 21998, 'double': 11934, 'fried': 15544, 'full': 15728, 'grease': 16885, 'asked': 3205, 'server': 40441, 'fresh': 15507, 'ones': 26582, 'said': 39419, 'fry': 15675, 'morning': 24152, 'warm': 48582, 'took': 45688, 'picking': 31199, 'put': 35855, 'fryer': 15679, 'gross': 17066, 'yuk': 49843, 'salsa': 39493, 'old': 26242, 'setting': 40550, 'got': 16705, 'every': 13535, 'one': 26430, 'started': 42842, 'fermenting': 14390, 'real': 37112, 'lack': 20996, 'caring': 6885, 'anymore': 2735, 'sad': 39372, 'oh': 26013, 'wo': 49317, 'nt': 25390, 'better': 4801, 'fresher': 15513, 'sushi': 43832, 'supermarket': 43703, 'overpriced': 28132, 'completely': 8743, 'poor': 33104, 'quality': 36010, 'pretty': 34345, 'sure': 43766, 'gave': 16046, 'us': 47730, 'canned': 6707, 'tuna': 46811, 'crab': 9575, 'soaked': 41882, 'mayo': 22965, 'service': 40452, 

In [49]:
X_train = bow_transformer.transform(X_train.values.astype('U'))
X_valid = bow_transformer.transform(X_valid.values.astype('U'))
X_test = bow_transformer.transform(X_test.values.astype('U'))

In [60]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=101)
#X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=101)


In [50]:
from sklearn.naive_bayes import MultinomialNB

X_full = sp.vstack((X_train , X_valid))
y_full = np.append(y_train, y_valid)

nb = MultinomialNB()
nb.fit(X_full, y_full)
y_hat = nb.predict(X_test)

print(accuracy_score(y_test, y_hat))

0.5863478987583572


In [23]:
#SVM
def linSVM(parameters, X, T, fold) : 
    n_folds = 3
    svc = LinearSVC(multi_class='ovr', dual=False)
    svc_cv = GridSearchCV(svc, parameters, cv=fold, scoring="accuracy", refit=False)
    svc_cv.fit(X, T)
    scores = svc_cv.cv_results_['mean_test_score']
    params = svc_cv.cv_results_['params']
    print('scores:',scores)
   #print('params:',params)
    return svc_cv.best_params_



def SVM(parameters, X, T, fold) : 
    n_folds = 3
    svmm = svm.SVC()
    svm_cv = GridSearchCV(svmm, parameters, cv=fold, scoring="accuracy", refit=False)
    svm_cv.fit(X, T)
    scores = svm_cv.cv_results_['mean_test_score']
    params = svm_cv.cv_results_['params']
    print('scores:',scores)
    print('params:',params)
    return svm_cv.best_params_

#placeX = X_valid
#placey = y_valid
#X_valid = X_test
#y_valid = y_test
#X_test = placeX
#y_test = placey

#Predefined Split for CV
my_test_fold = []
for i in range(X_train.shape[0]):
    my_test_fold.append(-1)
for i in range(X_valid.shape[0]):
    my_test_fold.append(0)

fold = PredefinedSplit(test_fold=my_test_fold)
CV_split_T = np.append(y_train, y_valid)

CV_split_X = sp.vstack((X_train , X_valid))  # NOT np.vstack


print(CV_split_T.shape)
print(CV_split_X.shape)


(301514,)
(301514, 50000)


In [26]:
Cs = [ 0.0001, 0.001, 0.01, 0.1, 1]
Cs = [ .002, .003, .004, .005, .006, .007, .008, .009]
Cs = [.0045, .0047, .0049, .0051, .0053, .0055, ]
Cs = [.0054, .0056, .0057, .0057, .0059 ]

param_grid = {'C': Cs}

bestParams = linSVM(param_grid, CV_split_X, CV_split_T, fold)

#svc = LinearSVC(C = .00215, multi_class='ovr')
#svc.fit(X_train, y_train)

#y_hat = svc.predict(X_test)
#print(accuracy_score(y_test, y_hat))

KeyboardInterrupt: 

In [ ]:
Cs = [ 0.001, 0.01, 0.1, 1, 10]
gammas = [ 0.001, 0.01, 0.1, 1]
param_grid = {'C': Cs, 'gamma': gammas}

bestParams = SVM(param_grid, CV_split_X, CV_split_T, fold)

#svm = svm.SVC(C = .003151, multi_class='ovr', dual=False)
#svc.fit(X_train, y_train)

#y_hat = svc.predict(X_test)
#print(accuracy_score(y_test, y_hat))

In [17]:


print(confusion_matrix(y_test, y_hat))
print('\n')
print(classification_report(y_test, y_hat))

print(accuracy_score(y_test, y_hat))

[[2020  291  125  162  261]
 [ 672  840  632  516  304]
 [ 201  386 1276 2244  653]
 [  93   97  509 6099 4184]
 [  57   19   91 2397 9375]]


             precision    recall  f1-score   support

          1       0.66      0.71      0.68      2859
          2       0.51      0.28      0.37      2964
          3       0.48      0.27      0.35      4760
          4       0.53      0.56      0.54     10982
          5       0.63      0.79      0.70     11939

avg / total       0.57      0.59      0.57     33504

0.585303247373448
